In [ ]:
from vit import utils, vit, our_vit

In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout, Activation, Input, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import cifar10
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import gc

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
(train_data, train_label), (test_data, test_label) = cifar10.load_data()
train_label = to_categorical(train_label)
test_label = to_categorical(test_label)
train_data = (train_data/255.).astype("float32")
test_data = (test_data/255.).astype("float32")

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, random_state=seed, shuffle=True,train_size=0.9)

In [ ]:
batch_size = 16
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.25, height_shift_range=0.25, zoom_range=0.25, horizontal_flip=True)
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)

In [ ]:
checkpoint_vit = ModelCheckpoint(r'C:\Users\fano\Desktop\weights\vit4_6.h5', 
    verbose=1, 
    monitor='val_loss',
    save_best_only=True, 
    mode='auto'
)  

In [ ]:
model = our_vit.ViT(image_size=  32, 32
                   patch_size=32,     4
                   num_classes=10
                   hidden_size=768,  512
                   num_layers=  6
                   num_heads=12,    12
                   mlp_dim=3072,    512
                   dropout=0.1,
                   emb_dropout=0
                   )
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()
gc.collect()

In [ ]:
plateau = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, verbose=1)

model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

history = model.fit(train_generator,
                    epochs=50,
                    validation_data=(X_valid, y_valid)
                   )
print("\nTest Accuracy: ", accuracy_score(np.argmax(test_label, axis=1), np.argmax(model.predict(test_data), axis=1)))

In [ ]:
# save the model !!!

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
results = model.evaluate(test_data,test_label, batch_size=32, verbose=1)
print("test loss, test acc:", results)